In [125]:
import urllib3
from urllib.parse import urlencode
import bs4
from bs4 import BeautifulSoup
import mysql.connector

In [150]:
URL = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev?serviceKey=b2AXFuhIJNJ13Rycl5fOfm3TLx3ouEbSlBwpAXEgNmZby%2FLAvMvOwdxDZCHiB2%2B34tNwOLcyMwqXA0Vs6mFvEA%3D%3D&'
DISTRICT = [11680, 11740, 11305, 11500, 11620, 11215, 11530, 11545, 11350, 11320, 11230, 11590, 11440, 11410, 11650, 11200, 11290, 11710, 11470, 11560, 11170, 11380, 11110, 11140, 11260]

def main():

    mydb = mysql.connector.connect(
      host="localhost",
      user="sungkyun",
      password="chung741",
      database="sungkyun"
    )
    mycursor = mydb.cursor()

    getData(mydb, mycursor)

In [145]:
def getData(mydb, mycursor):
    info = '?pageNo=1&numOfRows=10000&LAWD_CD=11680&DEAL_YMD='
    http = urllib3.PoolManager()
    for dist in DISTRICT:
        for year in range(2006, 2022):
            print(str(dist) + ':' + str(year))
            for month in range(1, 13):
                if month < 10:
                    ymd = str(year) + '0' + str(month)
                else:
                    ymd = str(year) + str(month)
                info = URL + '?pageNo=1&numofRows=10000&LAWD_CD=' + str(dist) + '&DEAL_YMD=' + ymd
                resp = http.request("GET", info)
                xml_obj = bs4.BeautifulSoup(resp.data,'lxml-xml')
                insert(xml_obj, mydb, mycursor)

In [149]:
def insert(resp, mydb, mycursor):
    rows = []
    for i in resp.findAll('item'):
        price = i.거래금액.text.replace(',', '')
        built_years = i.건축년도.text
        if int(i.월.text) < 10:
            sold_date =  i.년.text+ '-0' + i.월.text
        else:
            sold_date =  i.년.text + '-' + i.월.text
        if int(i.일.text) < 10:
            sold_date = sold_date + '-0' + i.일.text
        else:
            sold_date = sold_date + '-' + i.일.text
        print(i.도로명)
        street_adress = i.도로명.text
        dong = i.법정동.text
        name = i.아파트.text
        size = i.전용면적.text
        floor = i.층.text
        rows.append((price, built_years, sold_date, street_adress, dong, name, size, floor)) 
    
    sql = 'INSERT INTO apart_price (price, built_years, sold_date, street_adress, dong, name, size, floor) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'
    mycursor.executemany(sql, rows)
    mydb.commit()
    print(mycursor.rowcount, "was inserted.")
    
    

In [151]:
main()

11680:2006
<도로명>선릉로69길</도로명>
<도로명>선릉로69길</도로명>
<도로명>언주로90길</도로명>
<도로명>선릉로69길</도로명>
<도로명>언주로85길</도로명>
<도로명>역삼로</도로명>
<도로명>선릉로85길</도로명>
<도로명>역삼로</도로명>
None


AttributeError: 'NoneType' object has no attribute 'text'

In [38]:
url = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTradeDev'
headers={'serviceKey': 'b2AXFuhIJNJ13Rycl5fOfm3TLx3ouEbSlBwpAXEgNmZby/LAvMvOwdxDZCHiB2+34tNwOLcyMwqXA0Vs6mFvEA==', 'LAWD_CD': '11110', 'DEAL_YMD': '201512'}
http = urllib3.PoolManager()
resp = http.request("GET", url, headers=headers)
print(resp.data)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>'


In [89]:
xml_obj = bs4.BeautifulSoup(resp.data,'lxml-xml')

In [93]:
xml_obj.findAll('item')[2].거래금액.text

[<item><거래금액>    31,000</거래금액><건축년도>2002</건축년도><년>2021</년><도로명>언주로</도로명><도로명건물본번호코드>00427</도로명건물본번호코드><도로명건물부번호코드>00000</도로명건물부번호코드><도로명시군구코드>11680</도로명시군구코드><도로명일련번호코드>01</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>3005086</도로명코드><법정동> 역삼동</법정동><법정동본번코드>0720</법정동본번코드><법정동부번코드>0025</법정동부번코드><법정동시군구코드>11680</법정동시군구코드><법정동읍면동코드>10100</법정동읍면동코드><법정동지번코드>1</법정동지번코드><아파트>대우디오빌</아파트><월>1</월><일>5</일><일련번호>11680-275</일련번호><전용면적>30.03</전용면적><지번>720-25</지번><지역코드>11680</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item>,
 <item><거래금액>    61,000</거래금액><건축년도>2002</건축년도><년>2021</년><도로명>도곡로57길</도로명><도로명건물본번호코드>00009</도로명건물본번호코드><도로명건물부번호코드>00007</도로명건물부번호코드><도로명시군구코드>11680</도로명시군구코드><도로명일련번호코드>01</도로명일련번호코드><도로명지상지하코드>0</도로명지상지하코드><도로명코드>4166236</도로명코드><법정동> 역삼동</법정동><법정동본번코드>0766</법정동본번코드><법정동부번코드>0008</법정동부번코드><법정동시군구코드>11680</법정동시군구코드><법정동읍면동코드>10100</법정동읍면동코드><법정동지번코드>1</법정동지번코드><아파트>트레벨</아파트><월>1</월><일>6</일><일련번호>11680-274</일련번호><전용면적>33.48</전용면적><지번>766-8</지번><지역코드>11680</지역코드><층>3</층><해제사유발생

In [109]:
cols = ["price", "built_years", "sold_date", "street_adress", "dong", "name", "size", "floor"]
rows = []
  
for i in xml_obj.findAll('item'):
    price = i.거래금액.text.replace(',', '')
    built_years = i.건축년도.text
    if int(i.월.text) < 10:
        sold_date =  i.년.text+ '-0' + i.월.text
    else:
        sold_date =  i.년.text + '-' + i.월.text
    if int(i.일.text) < 10:
        sold_date = sold_date + '-0' + i.일.text
    else:
        sold_date = sold_date + '-' + i.일.text
    
    street_adress = i.도로명.text
    dong = i.법정동.text
    name = i.아파트.text
    size = i.전용면적.text
    floor = i.층.text
  
    rows.append({"price": price, "built_years": built_years, "sold_date": sold_date, "street_adress": street_adress
                 , "dong": dong, "name": name, "size": size, "floor": floor})
  
df = pd.DataFrame(rows, columns=cols)